## Deliverable 2. Create a Customer Travel Destinations Map.

In [144]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import gmaps
import sys
sys.path.append('..')
from config import weather_api_key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

from citipy import citipy

import requests
import time

In [107]:
# 1. Import the WeatherPy_database.csv file. 
WeatherPy_database_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
WeatherPy_database_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
0,0,Tiksi,RU,71.6872,128.8694,-11.99,93,100,13.42,overcast clouds
1,1,Yumen,CN,40.2833,97.2000,23.23,42,100,12.37,overcast clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,75.22,69,0,14.97,clear sky
3,3,Jamestown,US,42.0970,-79.2353,49.32,65,100,12.66,overcast clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,83,75,5.75,broken clouds


In [108]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [109]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = WeatherPy_database_df.loc[(WeatherPy_database_df["Max Temp"] <= max_temp) & \
                                       (WeatherPy_database_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
2,2,Mar Del Plata,AR,-38.0023,-57.5575,75.22,69,0,14.97,clear sky
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,83,75,5.75,broken clouds
6,6,Pangai,TO,-19.8000,-174.3500,82.51,74,20,9.22,few clouds
7,7,Vaini,TO,-21.2000,-175.2000,82.56,89,20,9.22,few clouds
9,9,Coihaique,CL,-45.5752,-72.0662,66.81,30,40,10.36,scattered clouds
10,10,Dhidhdhoo,MV,6.8833,73.1000,81.00,74,100,9.06,overcast clouds
11,11,Mogadishu,SO,2.0371,45.3438,77.63,81,100,23.02,overcast clouds
14,14,Atuona,PF,-9.8000,-139.0333,78.10,78,23,17.98,few clouds
15,15,Kapaa,US,22.0752,-159.3190,78.12,71,0,10.36,clear sky
16,16,Luderitz,NaN,-26.6481,15.1594,77.00,46,9,18.01,clear sky


In [110]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                120
City                   120
Country                118
Lat                    120
Lng                    120
Max Temp               120
Humidity               120
Cloudiness             120
Wind Speed             120
weather Description    120
dtype: int64

In [111]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,weather Description
2,2,Mar Del Plata,AR,-38.0023,-57.5575,75.22,69,0,14.97,clear sky
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,83,75,5.75,broken clouds
6,6,Pangai,TO,-19.8000,-174.3500,82.51,74,20,9.22,few clouds
7,7,Vaini,TO,-21.2000,-175.2000,82.56,89,20,9.22,few clouds
9,9,Coihaique,CL,-45.5752,-72.0662,66.81,30,40,10.36,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
217,217,Weligama,LK,5.9667,80.4167,76.32,81,98,3.85,overcast clouds
219,219,Valparaiso,CL,-33.0393,-71.6273,69.44,71,40,14.97,scattered clouds
220,220,Piacabucu,BR,-10.4056,-36.4344,82.51,68,91,12.91,overcast clouds
221,221,Camacha,PT,33.0833,-16.3333,63.82,82,0,10.36,clear sky


In [112]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
2,Mar Del Plata,AR,75.22,clear sky,-38.0023,-57.5575,
4,Arraial Do Cabo,BR,73.36,broken clouds,-22.9661,-42.0278,
6,Pangai,TO,82.51,few clouds,-19.8000,-174.3500,
7,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
9,Coihaique,CL,66.81,scattered clouds,-45.5752,-72.0662,
10,Dhidhdhoo,MV,81.00,overcast clouds,6.8833,73.1000,
11,Mogadishu,SO,77.63,overcast clouds,2.0371,45.3438,
14,Atuona,PF,78.10,few clouds,-9.8000,-139.0333,
15,Kapaa,US,78.12,clear sky,22.0752,-159.3190,
16,Luderitz,NaN,77.00,clear sky,-26.6481,15.1594,


In [116]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
      
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("Hotel not found... skipping.") 


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [117]:
hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
2,Mar Del Plata,AR,75.22,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Arraial Do Cabo,BR,73.36,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
6,Pangai,TO,82.51,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort
7,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Coihaique,CL,66.81,scattered clouds,-45.5752,-72.0662,Diego De Almagro Coyhaique
10,Dhidhdhoo,MV,81.00,overcast clouds,6.8833,73.1000,Dhidhoo beach
11,Mogadishu,SO,77.63,overcast clouds,2.0371,45.3438,Hotel Juba
14,Atuona,PF,78.10,few clouds,-9.8000,-139.0333,Villa Enata
15,Kapaa,US,78.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Luderitz,NaN,77.00,clear sky,-26.6481,15.1594,LÜDERITZ NEST HOTEL


In [126]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,weather Description,Lat,Lng,Hotel Name
2,Mar Del Plata,AR,75.22,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Arraial Do Cabo,BR,73.36,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
6,Pangai,TO,82.51,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort
7,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Coihaique,CL,66.81,scattered clouds,-45.5752,-72.0662,Diego De Almagro Coyhaique
10,Dhidhdhoo,MV,81.00,overcast clouds,6.8833,73.1000,Dhidhoo beach
11,Mogadishu,SO,77.63,overcast clouds,2.0371,45.3438,Hotel Juba
14,Atuona,PF,78.10,few clouds,-9.8000,-139.0333,Villa Enata
15,Kapaa,US,78.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Luderitz,NaN,77.00,clear sky,-26.6481,15.1594,LÜDERITZ NEST HOTEL


In [130]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [131]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [156]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig
# 

Figure(layout=FigureLayout(height='420px'))

In [162]:
#savefig("../Vacation_Search/WeatherPy_vacation_map.png")